## About Me 

This notebook is focused on using and querying TMDB efficiently 


Alternate idea: 
* Create database of all previously searched... no need to recall tmdb API --- can just search ones already archived 
- Not necessary right now

In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tmdbsimple as tmdb
import os
import sys
import pickle
import time

In [10]:
%load_ext autoreload
%autoreload 2

In [7]:
sys.path.append(os.path.abspath('../src'))

In [2]:
with open('../Data/api_key.pkl', 'rb') as hnd:
    tmdb.API_KEY = pickle.load(hnd)['api_key']

In [3]:
data = pd.read_csv('../Data/NetflixViewingHistory.csv')

In [9]:
import gather_data as eda1

In [13]:
netflix_df = eda1.parse_netflix(data)

Total number of TV Show + Movies 1405
TV Show vs Movie
TV Show    1291
Movie       114
Name: TV_Show_flag, dtype: int64
Dataframe shape:  (1405, 6)


In [19]:
shows = netflix_df[netflix_df['TV_Show_flag'] == 'TV Show']
movies = netflix_df[netflix_df['TV_Show_flag'] == 'Movie']

In [30]:
search= tmdb.Search()

## Start with Movies

In [22]:
movies.head()

,Title,Date,TV_Show_flag,Show Name,Season,Episode Name
3,Trevor Noah: Son of Patricia,2018-11-23,Movie,Trevor Noah,Son of Patricia,None
19,Captain Underpants: The First Epic Movie,2018-06-07,Movie,Captain Underpants,The First Epic Movie,None
66,Saving Capitalism,2017-12-07,Movie,Saving Capitalism,None,None
67,Betting on Zero,2017-12-07,Movie,Betting on Zero,None,None
75,Stranger Things: Stranger Things 2: Chapter Tw...,2017-11-11,Movie,Stranger Things,Stranger Things 2,"Chapter Two: Trick or Treat, Freak"


In [52]:
row1 = movies.iloc[0]
row1

Title           Trevor Noah: Son of Patricia
Date                     2018-11-23 00:00:00
TV_Show_flag                           Movie
Show Name                        Trevor Noah
Season                       Son of Patricia
Episode Name                            None
Name: 3, dtype: object

In [154]:
row2 = movies.iloc[6]
row2

Title           Boys Over Flowers: Volume 1: Episode 1
Date                               2017-09-23 00:00:00
TV_Show_flag                                     Movie
Show Name                            Boys Over Flowers
Season                                        Volume 1
Episode Name                                 Episode 1
Name: 110, dtype: object

In [53]:
search_results = search.movie(query=row1['Title'])
n_results = len(search_results['results'])
print(n_results)
temp_id = search_results['results'][0]['id']
full_movie_results = tmdb.Movies(temp_id)

1


In [54]:
full_movie_results.info()

{'adult': False,
 'backdrop_path': '/hDd2RWYR0mwGeF5oms5Ulr9zrhh.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 35, 'name': 'Comedy'}, {'id': 10770, 'name': 'TV Movie'}],
 'homepage': 'https://www.netflix.com/title/80239932',
 'id': 558341,
 'imdb_id': 'tt9170648',
 'original_language': 'en',
 'original_title': 'Trevor Noah: Son of Patricia',
 'overview': 'Trevor Noah gets out from behind the "Daily Show" desk and takes the stage for a stand-up special that touches on racism, immigration, camping and more.',
 'popularity': 3.145,
 'poster_path': '/dmhDeV3RYq4jMwgIMmo0W05uH8L.jpg',
 'production_companies': [],
 'production_countries': [{'iso_3166_1': 'US',
   'name': 'United States of America'}],
 'release_date': '2018-11-20',
 'revenue': 0,
 'runtime': 63,
 'spoken_languages': [{'iso_639_1': 'en', 'name': 'English'},
  {'iso_639_1': 'xh', 'name': ''}],
 'status': 'Released',
 'tagline': '',
 'title': 'Trevor Noah: Son of Patricia',
 'video': False,
 'vote_averag

In [55]:
normal_movie_fields = ['budget', 'genres', 'homepage', 'imdb_id', 'overview', 'popularity'\
                       , 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count']

In [56]:
set(normal_movie_fields).difference(set(full_movie_results.info().keys()))

set()

In [59]:
dir(full_movie_results)

['BASE_PATH',
 'URLS',
 '_DELETE',
 '_GET',
 '_POST',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_complete_url',
 '_get_credit_id_path',
 '_get_guest_session_id_path',
 '_get_id_path',
 '_get_params',
 '_get_path',
 '_get_series_id_season_number_episode_number_path',
 '_get_series_id_season_number_path',
 '_request',
 '_set_attrs_to_values',
 'account_states',
 'adult',
 'alternative_titles',
 'backdrop_path',
 'base_uri',
 'belongs_to_collection',
 'budget',
 'changes',
 'credits',
 'external_ids',
 'genres',
 'headers',
 'homepage',
 'id',
 'images',
 'imdb_id',
 'info',
 'keywords',
 'latest',
 'lists',
 'now_playing',
 'original_language',
 'original_titl

In [164]:
def get_movie_API_results(movie_title): 
        
    # Select requested fields from response 
    normal_movie_fields = ['budget', 'homepage', 'imdb_id', 'overview', 'popularity'\
                           , 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count']
    
    # Find the Movie in TMDB 
    search_results = search.movie(query=movie_title)
    n_results = len(search_results['results'])
#     print("N Results: ", n_results)
    if n_results == 0:
        movie_results = {key:np.nan for key in normal_movie_fields}
        movie_results['Number of Search Results'] = n_results
        movie_results['title_query'] = movie_title 
        return(movie_results)
    
    temp_id = search_results['results'][0]['id']
    full_movie_results = tmdb.Movies(temp_id)

    assert(set(normal_movie_fields).difference(set(full_movie_results.info().keys()))== set()), 'Movie result schema is missing a field'
    movie_results = {attr:getattr(full_movie_results, attr) for attr in normal_movie_fields}
    # TODO Fix genre parsing 
    
    
    # Append number of search results (incase there are multiple and we choose the wrong one)
    movie_results['Number of Search Results'] = n_results
    movie_results['title_query'] = movie_title
    
    
    time.sleep(0.6)
    return(movie_results)

In [162]:
r1 = get_movie_API_results(row1['Title'])
r1 

{'budget': 0,
 'homepage': 'https://www.netflix.com/title/80239932',
 'imdb_id': 'tt9170648',
 'overview': 'Trevor Noah gets out from behind the "Daily Show" desk and takes the stage for a stand-up special that touches on racism, immigration, camping and more.',
 'popularity': 3.145,
 'release_date': '2018-11-20',
 'revenue': 0,
 'runtime': 63,
 'vote_average': 7.1,
 'vote_count': 35,
 'Number of Search Results': 1,
 'title_query': 'Trevor Noah: Son of Patricia'}

In [155]:
r2 = get_movie_API_results(row2['Title'])
r2

{'budget': nan,
 'homepage': nan,
 'imdb_id': nan,
 'overview': nan,
 'popularity': nan,
 'release_date': nan,
 'revenue': nan,
 'runtime': nan,
 'vote_average': nan,
 'vote_count': nan,
 'title_query': 'Boys Over Flowers: Volume 1: Episode 1'}

In [156]:
demo_df = movies['Title'].iloc[:20].apply(get_movie_API_results)

In [157]:
demo_df.head()

3     {'budget': 0, 'homepage': 'https://www.netflix...
19    {'budget': 38000000, 'homepage': 'http://www.f...
66    {'budget': 0, 'homepage': 'http://www.netflix....
67    {'budget': 0, 'homepage': None, 'imdb_id': 'tt...
75    {'budget': nan, 'homepage': nan, 'imdb_id': na...
Name: Title, dtype: object

In [158]:
demo_df.to_list()

[{'budget': 0,
  'homepage': 'https://www.netflix.com/title/80239932',
  'imdb_id': 'tt9170648',
  'overview': 'Trevor Noah gets out from behind the "Daily Show" desk and takes the stage for a stand-up special that touches on racism, immigration, camping and more.',
  'popularity': 3.145,
  'release_date': '2018-11-20',
  'revenue': 0,
  'runtime': 63,
  'vote_average': 7.1,
  'vote_count': 35,
  'Number of Search Results': 1,
  'title_query': 'Trevor Noah: Son of Patricia'},
 {'budget': 38000000,
  'homepage': 'http://www.foxmovies.com/movies/captain-underpants-the-first-epic-movie',
  'imdb_id': 'tt2091256',
  'overview': 'Two mischievous kids hypnotize their mean elementary school principal and turn him into their comic book creation, the kind-hearted and elastic-banded Captain Underpants.',
  'popularity': 13.32,
  'release_date': '2017-06-01',
  'revenue': 125289450,
  'runtime': 89,
  'vote_average': 6.0,
  'vote_count': 564,
  'Number of Search Results': 1,
  'title_query': 'Cap

In [159]:
pd.DataFrame.from_dict(demo_df.to_list(), orient='columns')

,Number of Search Results,budget,homepage,imdb_id,overview,popularity,release_date,revenue,runtime,title_query,vote_average,vote_count
0,1.0,0.0,https://www.netflix.com/title/80239932,tt9170648,"Trevor Noah gets out from behind the ""Daily Sh...",3.145,2018-11-20,0.0,63.0,Trevor Noah: Son of Patricia,7.1,35.0
1,1.0,38000000.0,http://www.foxmovies.com/movies/captain-underp...,tt2091256,Two mischievous kids hypnotize their mean elem...,13.320,2017-06-01,125289450.0,89.0,Captain Underpants: The First Epic Movie,6.0,564.0
2,1.0,0.0,http://www.netflix.com/savingcapitalism,tt6185286,Former Secretary of Labor Robert Reich meets w...,1.610,2017-11-21,0.0,73.0,Saving Capitalism,7.1,17.0
3,1.0,0.0,None,tt3762912,Controversial hedge fund titan Bill Ackman is ...,2.099,2017-03-17,0.0,99.0,Betting on Zero,7.4,45.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stranger Things: Stranger Things 2: Chapter Tw...,NaN,NaN
5,1.0,100000.0,https://invisiblemoneydocumentary.wordpress.com/,tt5033790,Not since the invention of the Internet has th...,2.504,2016-12-30,0.0,90.0,Banking on Bitcoin,6.5,54.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Boys Over Flowers: Volume 1: Episode 1,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Disjointed: Part 1: Omega Strain,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marvel's The Defenders: Limited Series: The De...,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marvel's The Defenders: Limited Series: Fish i...,NaN,NaN


In [171]:
with open('../Data/all_movies_results_df.pkl', 'rb') as hnd:
    all_movies_results_df = pickle.load( hnd)

In [172]:
all_movies_results_df.head()

,Number of Search Results,budget,homepage,imdb_id,overview,popularity,release_date,revenue,runtime,title_query,vote_average,vote_count
0,1,0.0,https://www.netflix.com/title/80239932,tt9170648,"Trevor Noah gets out from behind the ""Daily Sh...",3.145,2018-11-20,0.0,63.0,Trevor Noah: Son of Patricia,7.1,35.0
1,1,38000000.0,http://www.foxmovies.com/movies/captain-underp...,tt2091256,Two mischievous kids hypnotize their mean elem...,13.320,2017-06-01,125289450.0,89.0,Captain Underpants: The First Epic Movie,6.0,564.0
2,1,0.0,http://www.netflix.com/savingcapitalism,tt6185286,Former Secretary of Labor Robert Reich meets w...,1.610,2017-11-21,0.0,73.0,Saving Capitalism,7.1,17.0
3,1,0.0,None,tt3762912,Controversial hedge fund titan Bill Ackman is ...,2.099,2017-03-17,0.0,99.0,Betting on Zero,7.4,45.0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stranger Things: Stranger Things 2: Chapter Tw...,NaN,NaN


In [173]:
all_movies_results_df.describe()

,Number of Search Results,budget,popularity,revenue,runtime,vote_average,vote_count
count,114.000000,4.500000e+01,45.000000,4.500000e+01,45.000000,45.000000,45.000000
mean,1.570175,3.360222e+07,12.477600,1.542482e+08,99.977778,6.802222,2826.111111
std,4.175295,5.757290e+07,12.566875,3.144806e+08,30.735299,1.348482,4509.053424
min,0.000000,0.000000e+00,0.600000,0.000000e+00,25.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,3.932000,0.000000e+00,87.000000,6.300000,56.000000
50%,0.000000,8.000000e+06,10.409000,3.142154e+06,99.000000,7.000000,564.000000
75%,1.000000,3.900000e+07,17.303000,1.610256e+08,120.000000,7.600000,4015.000000
max,20.000000,2.200000e+08,74.008000,1.519558e+09,165.000000,8.400000,18824.000000


In [176]:
# Missing Rate
all_movies_results_df.isna().sum()/all_movies_results_df.shape[0]

Number of Search Results    0.000000
budget                      0.605263
homepage                    0.815789
imdb_id                     0.614035
overview                    0.605263
popularity                  0.605263
release_date                0.605263
revenue                     0.605263
runtime                     0.605263
title_query                 0.000000
vote_average                0.605263
vote_count                  0.605263
dtype: float64